# Training Model for ASL Words

## Collecting Data And Splitting

To get all the data, I will loop through all the videos for all the words, and get the numpy arrays of each frame I saved during the pre-processing step

In [4]:
import numpy as np
import os

### Collecting The Data

In [8]:
num_words = 3
num_videos = 100
start_frames = 5
end_frames = 75
videos_base_dir = 'video_data'

Above, I defined to start getting the frames starting from the 5th frame. This is because I found out that the cv2 module keeps recording videos even when I put a waitKey(2000) to wait 2 seconds to get into position for each video. I was not able to find any solutions or reasons why this is happening. So, I will instead just disregard the first 5 frames of each video.

In [9]:
data = []
labels = []
for words in range(num_words):
    for videos in range(num_videos):
        video_array_list = []
        for frames in range(start_frames, end_frames):
            numpy_array_file_path = os.path.join(videos_base_dir,f'{words}',f'{videos}', 'landmarks', f'{words}_{videos}_{frames}.npy')
            temp_array = np.load(numpy_array_file_path)
            #print(f'Loading {numpy_array_file_path}')
            video_array_list.append(temp_array)
        data.append(video_array_list)
        labels.append(words)
        

To check if this worked, I will print the shape of the data and labels and also look at some of the data arrays

In [10]:
data_np = np.array(data)
print(f'Shape of data: {data_np.shape}')

labels_np = np.array(labels)
print(f'Shape of labels: {labels_np.shape}')

Shape of data: (300, 70, 1662)
Shape of labels: (300,)


The shape seems right. I will do some quick checks by grabbing random frames and checking with the data

In [ ]:
for word in range(num_words):
    for video in range(num_videos):
        for frame in range(start_frames, end_frames):
            if np.sum(data_np[video + (word*100)][frame-5] != np.load(os.path.join(videos_base_dir,f'{word}',f'{video}', 'landmarks', f'{word}_{video}_{frame}.npy'))):
                print(f'Error at {word}, {video}, {frame}')

In [19]:
print(labels)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


So this means all of the data was loaded successfully

### Randomly Splitting The Data

The next step is randomizing this data and then splitting it into 3 sections. Training data, Validation data, and Testing data.

In [22]:
rng = np.random.RandomState(113)
data_indices = np.arange(len(data))
#print(data_indices)
rng.shuffle(data_indices)

In [23]:
print(data_indices)

[151  79 268 198 177   6  92 269  34  12 233 144 175 259  32  33  40 231
 255  96 132  21  71 181  19  22 152 146  35 115 142 154 156 157 148 117
 272 133  62 141   3  56 143  63 221 147 182 210 145 260  46 298 294 257
 174  43 124  29 126 171  16  25  80  30  41  91 106  26 114 108  93  20
 158 120   0 119 212 219 134 299  44 168  82 229 234   5 105 286 129 247
 285 297 125  17  68 178  81 137 155 275   9 203  86  39  28 209 238 187
 251 192 250  94 195   8  78 283  24 236 256  42 131 226 169 123 293 190
 227 196 204 109  14 262  59 202  50  57 225  66 176 241 217   2 113  18
 200 159 166 264 103  13  51  84  58 107 237 140   4 121  48 104 111 205
  60 271 163 161  45 265  27 191 127 180  77  65 248 201  72  37 295  61
 110 130  98  53  99 223 214  75 232  10  54 164 128 179  90 292  52 188
 270 136 135 197 289  38 224 101  89 138 242 208 207 243 211 170 273 100
 239 263 206 118  49  15  76 216 153 194 291 112 235 172 160  87 290 193
  88  83 122 184 278 246  31 139 186  70 296 287 10

Now I have a randomized indices array. I can split this array into 3, and then get the data at these indices for each section. The split will be 70% training, 15% validation, and 15% testing

In [33]:
train_indices = data_indices[0:210]
val_indices = data_indices[210:255]
test_indices = data_indices[255:300]

In [34]:
x_train = data_np[train_indices]
y_train = labels_np[train_indices]

x_val = data_np[val_indices]
y_val = labels_np[val_indices]

x_test = data_np[test_indices]
y_test = labels_np[test_indices]

I will do some quick manual checks by looking at the randomized indices that were generated

In [44]:
print(x_train[5][5] == np.load('video_data/0/6/landmarks/0_6_10.npy'))
print(y_train[5])

[ True  True  True ...  True  True  True]
0


In [45]:
print(x_val[41][15] == np.load('video_data/2/82/landmarks/2_82_20.npy'))
print(y_val[41])


[ True  True  True ...  True  True  True]
2


In [46]:
print(x_test[44][50] == np.load('video_data/1/65/landmarks/1_65_55.npy'))
print(y_test[44])

[ True  True  True ...  True  True  True]
1


Ok it seems like the random shuffling has worked

## Creating And Training The Model

Now that I have training, testing, and validation data, I will create a model, compile it, and then train it using this data. Then I will run some predictions using the testing dataset.

### Creating A Model